In [1]:
!pip install --upgrade google-cloud-bigquery
!pip install pandas-gbq
!pip install torch torchvision torchaudio pandas scikit-learn matplotlib seaborn tqdm fuzz

In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
import os
import sys
import time
import json
import random
import traceback
from glob import glob
from datetime import datetime
from collections import OrderedDict

import numpy as np
import pandas as pd
from sklearn import metrics
from tqdm import tqdm

variable_map_dict = {
            # lab
            'WBC': 'wbc',
            'bun': 'bun',
            'sodium': 'sodium',
            'pt': 'pt',
            'INR': 'inr',
            'PTT': 'ptt',
            'platelet': 'platelet',
            'lactate' : 'lactate',
            'hemoglobin': 'hemoglobin',
            'glucose': 'glucose',
            'chloride': 'chloride',
            'creatinine': 'creatinine',
            'aniongap': 'aniongap',
            'bicarbonate': 'bicarbonate',

            # other lab
            'hematocrit': 'hematocrit',

            # used
            'heart rate': 'heartrate',
            'respiratory rate': 'resprate',
            'temperature': 'tempc',
            'meanbp': 'meanbp',
            'gcs': 'gcs_min',
            'urineoutput': 'urineoutput',
            'sysbp': 'sysbp',
            'diasbp': 'diasbp',
            'spo2': 'spo2',
            'Magnesium': 'magnesium',

            'C-reactive protein': 'c_reactive_protein',
            'bands': 'bands',
            }

item_id_dict = {
            'c_reactive_protein': '50889', #1298
            'magnesium': '50960', #203899
            }
id_item_dict = { v:k for k,v in item_id_dict.items() }

name_list = ['hadm_id', 'charttime']
for k,v in variable_map_dict.items():
    if k not in ['age', 'gender']:
        if len(v):
            name_list.append(v)
        elif k in item_id_dict:
            name_list.append(k)
name_index_dict = { name:id for id,name in enumerate(name_list) }



def time_to_second(t):
    t = str(t).replace('"', '').strip()

    # If time portion is missing, add "00:00:00"
    if " " not in t:
        t = t + " 00:00:00"
    else:
        date_part, time_part = t.split(" ", 1)
        parts = time_part.split(":")
        # Fill missing items with zeros
        while len(parts) < 3:
            parts.append("00")
        time_part = ":".join(parts[:3])
        t = f"{date_part} {time_part}"

    dt = datetime.strptime(t, "%Y-%m-%d %H:%M:%S")
    return int(time.mktime(dt.timetuple()))

In [4]:
def mywritejson(save_path,content):
    content = json.dumps(content,indent=4,ensure_ascii=False)
    with open(save_path,'w') as f:
        f.write(content)

def myreadjson(load_path):
    with open(load_path,'r') as f:
        return json.loads(f.read())

In [11]:
from google.cloud import bigquery
client = bigquery.Client(project='mimic-479113')
data_dir = "drive/MyDrive/bd4h"

In [5]:

query = """
SELECT *
FROM `physionet-data.mimiciii_clinical.patients`
"""

query_job = client.query(query)
patients_df = query_job.to_dataframe()

In [6]:
query = """
SELECT *
FROM `physionet-data.mimiciii_clinical.icustays`
"""

query_job = client.query(query)
icustays_df = query_job.to_dataframe()

In [7]:
query = """
SELECT *
FROM `physionet-data.mimiciii_clinical.diagnoses_icd`
"""

query_job = client.query(query)
diagnoses_icd_df = query_job.to_dataframe()

In [8]:
query = """
SELECT *
FROM `physionet-data.mimiciii_clinical.prescriptions`
"""

query_job = client.query(query)
prescriptions_df = query_job.to_dataframe()

In [6]:
query = """
SELECT *
FROM `physionet-data.mimiciii_clinical.labevents`
"""

query_job = client.query(query)
labevents_df = query_job.to_dataframe()

In [11]:
def generate_demo():
  patients_df['DOB'] = patients_df['DOB'].astype(str).str[:4].astype(int)

  # Ensure SUBJECT_ID is int and create dictionary
  sid_demo_dict = patients_df.set_index(patients_df['SUBJECT_ID'].astype(int))[['GENDER', 'DOB']].to_dict(orient='index')

  # Convert to [gender, dob] list
  sid_demo_dict = {k: [v['GENDER'], v['DOB']] for k, v in sid_demo_dict.items()}

  # Save JSON
  with open(os.path.join(data_dir, 'sid_demo_dict.json'), 'w') as f:
      json.dump(sid_demo_dict, f)

  df = icustays_df.merge(
    patients_df[['SUBJECT_ID', 'GENDER', 'DOB']],
    left_on='SUBJECT_ID', right_on='SUBJECT_ID', how='left'
  )

  # Compute age at ICU admission
  df['INTIME_YEAR'] = df['INTIME'].astype(str).str[:4].astype(int)
  df['AGE'] = df['INTIME_YEAR'] - df['DOB']

  # Filter patients <18, cap unrealistic ages
  df = df[df['AGE'] >= 18]
  df.loc[df['AGE'] > 150, 'AGE'] = 90
  icu_hadm_dict = myreadjson(os.path.join(data_dir, 'icu_hadm_dict.json'))
  icu_hadm_dict_int_keys = {int(k): v for k, v in icu_hadm_dict.items()}

  # reduces from 53362 to 11715
  # If you just need the keys as ints
  target_icu_hadm = icu_hadm_dict_int_keys.keys()

  df = df[df['ICUSTAY_ID'].isin(list(target_icu_hadm))]
  df['SUBJECT_ID'] = df['SUBJECT_ID'].astype(str)
  df['HADM_ID'] = df['HADM_ID'].astype(str)
  df['INTIME'] = df['INTIME'].astype(str)

  sid_hadm_dict = df.groupby('SUBJECT_ID')['HADM_ID'].apply(list).to_dict()

  # HADM_ID -> SUBJECT_ID
  hadm_sid_dict = df.set_index('HADM_ID')['SUBJECT_ID'].to_dict()

  # HADM_ID -> [gender, age]
  hadm_demo_dict = df.set_index('HADM_ID')[['GENDER', 'AGE']].apply(lambda x: x.tolist(), axis=1).to_dict()

  # HADM_ID -> ICU admission time
  hadm_time_dict = df.set_index('HADM_ID')['INTIME'].to_dict()

  # --- 4️⃣ Save JSONs ---
  for d, fname in zip(
      [hadm_demo_dict, hadm_time_dict, sid_hadm_dict, hadm_sid_dict],
      ['hadm_demo_dict.json', 'hadm_time_dict.json', 'sid_hadm_dict.json', 'hadm_sid_dict.json']
  ):
      with open(os.path.join(data_dir, fname), 'w') as f:
          json.dump(d, f)

  print("All dictionaries generated and saved to:", data_dir)

In [14]:
generate_demo()

All dictionaries generated and saved to: drive/MyDrive/bd4h


In [12]:
from collections import defaultdict
def generate_diagnosis_data():
  sid_hadm_dict = myreadjson(os.path.join(data_dir, 'sid_hadm_dict.json') )
  hadm_sid_dict = myreadjson(os.path.join(data_dir, 'hadm_sid_dict.json'))

  hadm_map_dict = dict()
  for hadm in hadm_sid_dict:
      sid = hadm_sid_dict[hadm]
      hadm_list = sid_hadm_dict[sid]
      if len(hadm_list) > 1:
          hadm_list = sorted(hadm_list, key=lambda k:int(k))
          idx = hadm_list.index(hadm)
          if idx > 0:
              for h in hadm_list[:idx]:
                  if h not in hadm_map_dict:
                      hadm_map_dict[h] = []
                  hadm_map_dict[h].append(int(hadm))

  hadm_icd_dict = defaultdict(list)

  # Filter only HADM_IDs that are in hadm_map_dict
  diagnoses_icd_df['HADM_ID_STR'] = diagnoses_icd_df['HADM_ID'].astype(str)
  diag_filtered = diagnoses_icd_df[diagnoses_icd_df["HADM_ID_STR"].isin(hadm_map_dict)]

  for _, row in diag_filtered.iterrows():
      hadm_id = row["HADM_ID_STR"]
      icd = row["ICD9_CODE"]

      for mapped_id in hadm_map_dict[hadm_id]:
          hadm_icd_dict[mapped_id].append(icd)

  hadm_icd_dict = {h: list(set(icds)) for h, icds in hadm_icd_dict.items()}

  # Save
  mywritejson(os.path.join(data_dir, "hadm_icd_dict.json"), hadm_icd_dict)

In [16]:
generate_diagnosis_data()

In [13]:
def generate_drug_data():
  hadm_sid_dict = myreadjson(os.path.join(data_dir, 'hadm_sid_dict.json'))
  hadm_id_set = set(hadm_sid_dict)
  prescriptions_df['HADM_ID_STR'] = prescriptions_df['HADM_ID'].astype(str)
  prescriptions_df['STARTDATE'] = prescriptions_df['STARTDATE'].astype(str)
  prescriptions_df['ENDDATE'] = prescriptions_df['ENDDATE'].astype(str)
  # Filter only relevant rows
  df = prescriptions_df[prescriptions_df["HADM_ID_STR"].isin(hadm_id_set)].copy()

  df["TIME_KEY"] = df["STARTDATE"] + " -- " + df["ENDDATE"]

  # Initialize nested dict
  hadm_time_drug_dict = {}

  # Iterate rows (still fast after filtering)
  for _, row in df.iterrows():
      hadm_id = row["HADM_ID"]
      time_key = row["TIME_KEY"]
      drug = row["DRUG"]

      if hadm_id not in hadm_time_drug_dict:
          hadm_time_drug_dict[hadm_id] = {}

      if time_key not in hadm_time_drug_dict[hadm_id]:
          hadm_time_drug_dict[hadm_id][time_key] = []

      hadm_time_drug_dict[hadm_id][time_key].append(drug)

  # Save output
  mywritejson(os.path.join(data_dir, "hadm_time_drug_dict.json"), hadm_time_drug_dict)

In [18]:
generate_drug_data()

In [15]:
def generate_ehr_files():
    hadm_time_dict = myreadjson(os.path.join(data_dir, 'hadm_time_dict.json'))
    hadm_demo_dict = myreadjson(os.path.join(data_dir, 'hadm_demo_dict.json'))
    hadm_sid_dict = myreadjson(os.path.join(data_dir, 'hadm_sid_dict.json'))
    hadm_icd_dict = myreadjson(os.path.join(data_dir, 'hadm_icd_dict.json'))
    hadm_time_drug_dict = myreadjson(os.path.join(data_dir, 'hadm_time_drug_dict.json'))
    groundtruth_dir = os.path.join(data_dir, 'train_groundtruth')
    mkdir(groundtruth_dir)
    ehr_count_dict = dict()

    for hadm_id in hadm_sid_dict:

        time_drug_dict = hadm_time_drug_dict.get(hadm_id, { })
        icd_list = hadm_icd_dict.get(hadm_id, [])
        demo = hadm_demo_dict[hadm_id]
        demo[0] = demo[0] + '1'
        demo[1] = 'A' + str(int(demo[1] / 9))
        icd_demo = icd_list + demo

        for icd in icd_demo:
            ehr_count_dict[icd] = ehr_count_dict.get(icd, 0) + 1

        ehr_dict = { 'drug':{ }, 'icd_demo': icd_demo}

        for setime, drug_list in time_drug_dict.items():
            try:
              stime,etime = setime.split(' -- ')
              start_second = time_to_second(hadm_time_dict[hadm_id])
              stime = str((time_to_second(stime) - start_second) / 3600)
              etime = str((time_to_second(etime) - start_second) / 3600)
              setime = stime + ' -- ' + etime
              for drug in drug_list:
                  ehr_count_dict[drug] = ehr_count_dict.get(drug, 0) + 1
              ehr_dict['drug'][setime] = list(set(drug_list))
            except:
                pass

        mywritejson(os.path.join(groundtruth_dir, hadm_id + '.json'), ehr_dict)
        # break
    mywritejson(os.path.join(data_dir, 'ehr_count_dict.json'), ehr_count_dict)

In [21]:
generate_ehr_files()

In [16]:
def map_ehr_id():
    ehr_count_dict = myreadjson(os.path.join(data_dir, 'ehr_count_dict.json'))
    ehr_list = [ehr for ehr,c in ehr_count_dict.items() if c > 100]
    ns = set('0123456789')
    drug_list = [e for e in ehr_list if e[1] in ns]
    med_list = [e for e in ehr_list if e[1] not in ns]
    mywritejson(os.path.join(data_dir, 'ehr_list.json'), ehr_list)

In [25]:
map_ehr_id()

In [51]:
def select_records_of_variables_not_in_pivoted():
  count_dict = { v:0 for v in item_id_dict.values() }
  hadm_time_dict = myreadjson(os.path.join(data_dir,'hadm_time_dict.json' ))
  #hadm_time_dict has 11715 records # every time in icu aswell
  #lab events had 27m+ filtered to 205197
  labevents_df['HADM_ID_STR'] = labevents_df['HADM_ID'].astype(str)
  labevents_df['ITEMID_STR'] = labevents_df['ITEMID'].astype(str)

  filtered_df = labevents_df[
      labevents_df["HADM_ID_STR"].isin(hadm_time_dict.keys()) &
      labevents_df["ITEMID_STR"].isin(count_dict.keys())
  ].copy()
  output_path = os.path.join(data_dir, "sepsis_lab.csv")
  filtered_df.to_csv(output_path, index=False)

  print("Done. Filtered rows:", len(filtered_df))

In [52]:
select_records_of_variables_not_in_pivoted()

Done. Filtered rows: 205197


In [53]:
def generate_variables_not_in_pivoted():
  import pandas as pd
  id_item_dict = {v: k for k, v in item_id_dict.items()}

  # ordered list of item names
  head = sorted(item_id_dict)

  # initialize counts
  count_dict = {v: 0 for v in item_id_dict.values()}

  # load sepsis_lab.csv (same columns as original file)
  df = pd.read_csv(os.path.join(data_dir, "sepsis_lab.csv"))

  # expected: df columns include: hadm_id, item_id, charttime, value
  # convert value to numeric, drop non-numeric rows
  df["VALUE"] = pd.to_numeric(df["VALUE"], errors="coerce")
  df = df.dropna(subset=["VALUE"])

  df['ITEMID_STR'] = df['ITEMID'].astype(str)
  df["ITEM_NAME"] = df["ITEMID_STR"].map(id_item_dict)

  # PIVOT: rows = (hadm_id, charttime), columns = item_name
  pivot_df = df.pivot_table(
      index=["HADM_ID", "CHARTTIME"],
      columns="ITEM_NAME",
      values="VALUE",
      aggfunc="first"      # one value per timepoint
  )

  # make sure all head columns appear, in correct order
  pivot_df = pivot_df.reindex(columns=head)

  # restore index as columns
  pivot_df = pivot_df.reset_index()

  # save output
  pivot_df.to_csv(os.path.join(data_dir, "pivoted_add.csv"), index=False)


In [54]:
generate_variables_not_in_pivoted()

/tmp/ipython-input-4292233960.py:12: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(data_dir, "sepsis_lab.csv"))


In [7]:
query = """
SELECT *
FROM `physionet-data.mimiciii_derived.pivoted_sofa`
"""

query_job = client.query(query)
pivoted_sofa_df = query_job.to_dataframe()

In [8]:
query = """
SELECT *
FROM `physionet-data.mimiciii_derived.pivoted_lab`
"""

query_job = client.query(query)
pivoted_lab_df = query_job.to_dataframe()

In [9]:
query = """
SELECT *
FROM `physionet-data.mimiciii_derived.pivoted_vital`
"""

query_job = client.query(query)
pivoted_vital_df = query_job.to_dataframe()

In [10]:
pivoted_add_df = pd.read_csv(os.path.join(data_dir,'pivoted_add.csv' ))

In [16]:
# cleanup
hadm_time_dict = myreadjson(os.path.join(data_dir, 'hadm_time_dict.json' ))
icu_hadm_dict = myreadjson(os.path.join(data_dir, 'icu_hadm_dict.json' ))
merge_dir = os.path.join("merged")
pivoted_dir = os.path.join("pivoted_sofa")
print(merge_dir)
os.system('rm -r ' + merge_dir)
os.system('mkdir ' + merge_dir)
os.system('rm -r ' + pivoted_dir)
os.system('mkdir ' + pivoted_dir)

merged


0

In [71]:
pivoted_vital_df.head()

,icustay_id,charttime,HeartRate,SysBP,DiasBP,MeanBP,RespRate,TempC,SpO2,Glucose
0,226013,2154-11-10 06:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0
1,226267,2121-09-28 03:46:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.0
2,226321,2109-06-07 21:02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.0
3,227130,2135-06-18 01:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.0
4,227518,2147-09-01 07:45:00,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN


In [17]:

def merge_pivoted_data(csv_list):
  hadm_time_dict = myreadjson(os.path.join(data_dir, 'hadm_time_dict.json' ))
  icu_hadm_dict = myreadjson(os.path.join(data_dir, 'icu_hadm_dict.json' ))
  merge_dir = os.path.join("merged")
  pivoted_dir = os.path.join("pivoted_sofa")
  for fi in csv_list:
    if fi=="pivoted_sofa":
      df = pivoted_sofa_df
    elif fi == "pivoted_add":
      df = pivoted_add_df
      df = df.rename(columns={"HADM_ID": "hadm_id", "CHARTTIME":"charttime"})
    elif fi == "pivoted_lab":
      df = pivoted_lab_df
    else:
      df = pivoted_vital_df

    df.columns = df.columns.str.replace('"', '').str.strip()

    head = df.columns.tolist()                # same as your old "head"
    if fi == 'pivoted_sofa':
        sofa_head = ",".join(['time'] + head[4:])

    i=0
    for _, row in df.iterrows():
        line_dict = {col: row[col] for col in head if pd.notna(row[col])}
        i = i+1
        if i%100000 ==0:

          print(i , " processed")

        # -------------------------------------------------------------
        # Special processing for pivoted_sofa.csv
        # -------------------------------------------------------------
        if fi == 'pivoted_sofa':
          icu_id = str(line_dict.get('icustay_id', 'xxx'))
          if str(icu_id) not in icu_hadm_dict:
            continue
          hadm_id = str(icu_hadm_dict[icu_id])
          line_dict['hadm_id'] = hadm_id
          line_dict['charttime'] = line_dict['starttime']
        hadm_id = str(line_dict.get('hadm_id', 'xxx'))


        if hadm_id not in hadm_time_dict:
          icu_id = str(line_dict.get('icustay_id', 'xxx'))
          hadm_id = str(icu_hadm_dict.get(icu_id,'xxx'))
          if hadm_id not in hadm_time_dict:
            continue

        # -------------------------------------------------------------
        # Compute delta_hour
        # -------------------------------------------------------------
        hadm_time = time_to_second(hadm_time_dict[hadm_id])
        now_time = time_to_second(line_dict['charttime'])
        delta_hour = int((now_time - hadm_time) / 3600)
        line_dict['charttime'] = str(delta_hour)


        if fi == 'pivoted_sofa':
            sofa_file = os.path.join(pivoted_dir, hadm_id + '.csv')

            if not os.path.exists(sofa_file):
                with open(sofa_file, 'w') as f:
                    f.write(sofa_head + "\n")

            # sofa line = [delta_hour] + row[4:]
            sofa_line = [str(delta_hour)] + [str(row[h]) for h in head[4:]]
            with open(sofa_file, 'a') as wf:
                wf.write(",".join(sofa_line) + "\n")

        line_dict = {k.lower(): v for k, v in line_dict.items()}
        line_dict["hadm_id"] = hadm_id

        new_line = [str(line_dict.get(name, "")) for name in name_list]
        new_line = ",".join(new_line) + "\n"

        hadm_file = os.path.join(merge_dir, hadm_id + '.csv')
        if not os.path.exists(hadm_file):
            with open(hadm_file, 'w') as f:
                f.write(",".join(name_list) + '\n')

        with open(hadm_file, 'a') as wf:
            wf.write(new_line)


In [26]:
pivoted_add_df.shape

(204194, 4)

In [27]:
csv_list = ['pivoted_sofa', 'pivoted_add', 'pivoted_lab', 'pivoted_vital']
# pivoted_sofa_df.shape
merge_pivoted_data(csv_list)

100000  processed
200000  processed


In [34]:
import os
from tqdm import tqdm

def sort_and_merge_files():
    """Sorts pivoted data files by time intervals and merges lines."""

    sorted_dir = 'sort_pivoted'
    merged_dir = 'merged'

    # Clear and recreate the sorted directory
    if os.path.exists(sorted_dir):
        os.system(f'rm -r {sorted_dir}')
    os.makedirs(sorted_dir, exist_ok=True)

    for filename in tqdm(os.listdir(merged_dir), desc="Processing files"):
        input_path = os.path.join(merged_dir, filename)
        output_path = os.path.join(sorted_dir, filename)

        time_dict = {}
        with open(input_path, 'r') as infile, open(output_path, 'w') as outfile:
            for i, line in enumerate(infile):
                parts = line.strip().split(',')

                # Handle header
                if i == 0:
                    header = parts[1:]  # Skip first column
                    header[0] = 'time'
                    outfile.write(','.join(header) + '\n')
                    continue

                # Group lines by rounded time intervals
                delta = 3
                time_key = delta * (int(parts[1]) // delta)
                time_dict.setdefault(time_key, []).append(parts)

            # Merge lines for each time interval
            for t in sorted(time_dict):
                merged_line = time_dict[t][0]
                for line_parts in time_dict[t][1:]:
                    for idx, value in enumerate(line_parts):
                        if value.strip():
                            merged_line[idx] = value
                # Write merged line, skipping the first column
                outfile.write(','.join(merged_line[1:]) + '\n')

In [35]:
sort_and_merge_files()

100%|██████████| 11715/11715 [00:17<00:00, 661.41it/s]


In [25]:

os.system('rm -r ./train_groundtruth/*.csv')
os.system('rm -r ./train_groundtruth')
os.system('rm -r ./train_with_missing')
os.mkdir('./train_groundtruth')
os.mkdir('./train_with_missing')
os.system('cp -r drive/Mydrive/bd4h/sort_pivoted/* ./train_groundtruth1/')
os.system('cp -r ./train_groundtruth1/* ./train_groundtruth/')

0

In [26]:
# this is from original repository
import os
import numpy as np
from glob import glob
from tqdm import tqdm

def generate_lab_missing_values():
    """Generates lab data with simulated missing values."""

    lab_files = glob(os.path.join('train_groundtruth', '*.csv'))
    output_dir = 'train_with_missing'

    # Clear the output directory
    if os.path.exists(output_dir):
        for f in os.listdir(output_dir):
            os.remove(os.path.join(output_dir, f))
    else:
        os.makedirs(output_dir)

    feat_count_dict = {}
    line_count_dict = {}
    n_full_files = 0

    for file_path in tqdm(lab_files, desc="Processing lab files"):
        file_data = []
        valid_flags = []
        last_row_data = [-10000]

        with open(file_path, 'r') as f:
            for line_idx, line in enumerate(f):
                parts = line.strip().split(',')

                # Handle header
                if line_idx == 0:
                    feat_list = parts
                    vector = [0] * len(feat_list)
                    for feat in feat_list:
                        feat_count_dict.setdefault(feat, [0, 0])
                    file_data.append(parts)
                    valid_flags.append([1] * len(parts))
                    continue

                # Skip invalid time points
                time_val = int(parts[0])
                if time_val < -24 or time_val >= 500:
                    continue
                assert time_val > -200

                # Track valid features and counts
                valid_row = []
                for idx, val in enumerate(parts):
                    feat_count_dict[feat_list[idx]][0] += 1
                    if val in ['', 'NA']:
                        feat_count_dict[feat_list[idx]][1] += 1
                        valid_row.append(0)
                    else:
                        valid_row.append(1)
                        vector[idx] = 1

                # Merge rows with the same timestamp
                if parts[0] == last_row_data[0]:
                    for idx, val in enumerate(parts):
                        if valid_row[idx]:
                            last_row_data[idx] = val
                            valid_flags[-1][idx] = valid_row[idx]
                    file_data[-1] = last_row_data
                else:
                    file_data.append(parts)
                    valid_flags.append(valid_row)
                    last_row_data = parts

        # Update line counts
        line_count_dict[len(file_data)] = line_count_dict.get(len(file_data), 0) + 1

        # Check how many features are fully observed
        feature_counts = [sum(1 for row in file_data[1:] if row[idx].strip() not in ['', 'NA'])
                          for idx in range(len(file_data[0]))]
        if min(feature_counts) >= 2:
            n_full_files += 1

        # Skip small files or files with insufficient data
        if len(file_data) < 5 or sorted(vector)[2] < 1:
            os.remove(file_path)
            continue

        # Save cleaned ground truth
        with open(file_path, 'w') as f:
            f.write('\n'.join([','.join(row) for row in file_data]))

        # Introduce random missing values
        valid_flags = np.array(valid_flags)
        valid_flags[0] = 0  # Header row is always valid
        for col_idx in range(1, valid_flags.shape[1]):
            valid_indices = np.where(valid_flags[:, col_idx] > 0)[0]
            if len(valid_indices) > 2:
                indices_to_remove = valid_indices[1:-1]
                np.random.shuffle(indices_to_remove)
                file_data[indices_to_remove[0]][col_idx] = ''

        # Save data with missing values
        output_path = os.path.join(output_dir, os.path.basename(file_path))
        with open(output_path, 'w') as f:
            f.write('\n'.join([','.join(row) for row in file_data]))

    print(f"Number of full files: {n_full_files}")


In [27]:
generate_lab_missing_values()

100%|██████████| 11715/11715 [00:16<00:00, 722.69it/s]

37


In [39]:

import sys
import os
import sys
import time
import numpy as np
from sklearn import metrics
import random
import json
from glob import glob
from collections import OrderedDict
from tqdm import tqdm


def generate_feature_mm_dict():
    files = sorted(glob(os.path.join('train_groundtruth/*')))
    feature_value_dict = dict()
    for ifi, fi in enumerate(tqdm(files)):
        if 'csv' not in fi:
            continue
        for iline, line in enumerate(open(fi)):
            line = line.strip()
            if iline == 0:
                feat_list = line.split(',')
            else:
                data = line.split(',')
                for iv, v in enumerate(data):
                    if v in ['NA', '']:
                        continue
                    else:
                        feat = feat_list[iv]
                        if feat not in feature_value_dict:
                            feature_value_dict[feat] = []
                        feature_value_dict[feat].append(float(v))
    feature_mm_dict = dict()
    feature_ms_dict = dict()

    feature_range_dict = dict()
    for feat, vs in feature_value_dict.items():
        vs = sorted(vs)
        value_split = []
        for i in range(4000):
            n = int(i * len(vs) / 4000)
            value_split.append(vs[n])
        value_split.append(vs[-1])
        feature_range_dict[feat] = value_split


        n = int(len(vs) / 4000)
        feature_mm_dict[feat] = [vs[n], vs[-n - 1]]
        feature_ms_dict[feat] = [np.mean(vs), np.std(vs)]

    mywritejson(os.path.join(data_dir, 'MIMIC_feature_mm_dict.json'), feature_mm_dict)
    mywritejson(os.path.join(data_dir, 'MIMIC_feature_ms_dict.json'), feature_ms_dict)
    mywritejson(os.path.join(data_dir, 'MIMIC_feature_list.json'), feat_list)
    mywritejson(os.path.join(data_dir, 'MIMIC_feature_value_dict_{:d}.json'.format(4000)), feature_range_dict)



In [40]:
generate_feature_mm_dict()

100%|██████████| 11713/11713 [00:04<00:00, 2587.31it/s]


In [41]:
def split_data_to_ten_set():
    files = sorted(glob(os.path.join('train_with_missing/*')))
    np.random.shuffle(files)
    splits = []
    for i in range(10):
        st = int(len(files) * i / 10)
        en = int(len(files) * (i+1) / 10)
        splits.append(files[st:en])
    mywritejson(os.path.join(data_dir, 'MIMIC_splits.json'), splits)


In [42]:
split_data_to_ten_set()

This is Auto Encoder Module

In [43]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

def create_value_embedding(embed_dim=512, split_size=200):
    """Generate sinusoidal value embeddings."""
    half_dim = embed_dim // 2
    vals = np.array([np.arange(split_size) * i for i in range(half_dim)], dtype=np.float32).T
    vals /= vals.max()
    embeddings = np.concatenate([np.sin(vals), np.cos(vals)], axis=1)
    embeddings[0, :embed_dim] = 0
    return torch.from_numpy(embeddings)


class NeuralNet(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.args = args

        # Embedding layer setup
        self.embedding = nn.Embedding(args.vocab_size, args.embed_size)
        self.lstm = nn.LSTM(
            input_size=args.embed_size,
            hidden_size=args.hidden_size,
            num_layers=args.num_layers,
            batch_first=True,
            bidirectional=args.brnn
        )

        self.dd_embedding = nn.Embedding(args.n_ehr, args.embed_size)
        self.value_embedding = nn.Embedding.from_pretrained(
            create_value_embedding(args.embed_size, args.split_num + 1)
        )

        # Pre/post embedding and mapping blocks
        self.pre_embedding = self._make_mlp_block(args.embed_size * 2, args.embed_size)
        self.post_embedding = self._make_mlp_block(args.embed_size * 2, args.embed_size)
        self.pre_mapping = self._make_mlp_block(args.embed_size, args.embed_size)
        self.post_mapping = self._make_mlp_block(args.embed_size, args.embed_size)
        self.dd_mapping = self._make_mlp_block(args.embed_size, args.embed_size, dropout=0.1, double=True)
        self.value_mapping = self._make_mlp_block(args.embed_size * 2, args.embed_size, dropout=0.1)
        self.mapping = self._make_mlp_block(args.embed_size, args.embed_size)
        self.embed_linear = self._make_mlp_block(args.embed_size, args.embed_size, dropout=0.1)

        # LSTM output mapping
        lstm_size = args.rnn_size * 2 if args.brnn else args.rnn_size
        hidden_size = args.hidden_size
        self.tah_mapping = self._make_mlp_block(lstm_size, hidden_size, dropout=0.1)
        self.tav_mapping = self._make_mlp_block(hidden_size, hidden_size, dropout=0.1)
        self.output = self._make_mlp_block(lstm_size, args.rnn_size, dropout=0.1)
        self.value = self._make_mlp_block(hidden_size, args.output_size, dropout=0.1)
        self.pooling = nn.AdaptiveMaxPool1d(1)

    def _make_mlp_block(self, in_dim, out_dim, dropout=0.0, double=False):
        """Helper to create MLP blocks."""
        layers = [nn.Linear(in_dim, out_dim), nn.ReLU()]
        if dropout > 0:
            layers.append(nn.Dropout(dropout))
        if double:
            layers.extend([nn.Linear(out_dim, out_dim), nn.ReLU(), nn.Dropout(dropout)])
        return nn.Sequential(*layers)

    def visit_pooling(self, x):
        batch, visits, features, embed = x.size()
        x = x.view(batch * visits, features, embed).transpose(1, 2)
        x = self.pooling(x).view(batch, visits, embed)
        return x

    def value_order_embedding(self, x):
        index, value = x
        size = list(index[0].size())
        idx_emb = self.embedding(index.view(-1))
        val_emb = self.value_embedding(value.view(-1))
        combined = self.value_mapping(torch.cat([idx_emb, val_emb], dim=1))
        size.append(-1)
        return combined.view(size)

    def pp_value_embedding(self, neib):
        """Process pre/post embeddings for neighbors."""
        size = list(neib[1].size())
        if self.args.use_ve == 0:
            pre = self.pre_mapping(self.embedding(neib[0]))
            post = self.post_mapping(self.embedding(neib[2]))
        else:
            pre = self.value_order_embedding(neib[0])
            post = self.value_order_embedding(neib[2])
            pre_t = self.value_embedding(neib[1].view(-1)).view(size + [-1])
            post_t = self.value_embedding(neib[3].view(-1)).view(size + [-1])
            pre = self.pre_embedding(torch.cat([pre, pre_t], dim=3))
            post = self.post_embedding(torch.cat([post, post_t], dim=3))
        return pre, post

    def time_aware_attention(self, hidden, vectors):
        """Compute time-aware attention over vectors."""
        wh = self.tah_mapping(hidden).unsqueeze(2).expand_as(vectors)
        wv = self.tav_mapping(vectors)
        alpha = F.softmax(wh + wv, dim=2)
        alpha = alpha.transpose(2, 3).contiguous()
        vectors = vectors.transpose(2, 3).contiguous()
        batch_size = vectors.size(0) * vectors.size(1)
        att_res = torch.bmm(alpha.view(batch_size, 1, alpha.size(3)),
                            vectors.view(batch_size, vectors.size(3), 1))
        return att_res.view(*vectors.size()[:3])

    def forward(self, x, neib=None, dd=None, mask=None):
        # Input embedding
        if self.args.value_embedding == 'no':
            x = self.embedding(x)
        elif self.args.value_embedding == 'use_order':
            x = self.value_order_embedding(x)
        else:
            x = self.embedding(x.view(-1))

        # Domain-specific embeddings
        if dd is not None:
            dsize = list(dd.size()) + [-1]
            d_emb = self.dd_mapping(self.dd_embedding(dd.view(-1)).view(dsize))
            x = self.mapping(torch.cat([x, d_emb], dim=2))

        x = self.visit_pooling(x)

        # LSTM pass
        lstm_out, _ = self.lstm(x)
        out = self.output(lstm_out)

        if neib is not None:
            pre_x, post_x = self.pp_value_embedding(neib)
            pp = torch.cat([pre_x, post_x], dim=2)
            if self.args.use_ta:
                out += self.time_aware_attention(lstm_out, pp)
            else:
                out += self.visit_pooling(pp)

        return self.value(out)


Helper functions

In [1]:
import os
import numpy as np
import torch
from sklearn import metrics
from torch.autograd import Variable
from tqdm import tqdm

# ---------------------
# Normalized RMSE
# ---------------------
def normalized_rmse(predictions, targets, mask):
    assert predictions.shape == targets.shape == mask.shape

    missing = mask == 1
    observed = mask == 0

    rmse_missing = np.sqrt(((predictions[missing] - targets[missing]) ** 2).mean())
    rmse_observed = np.sqrt(((predictions[observed] - targets[observed]) ** 2).mean())

    metrics_list = [rmse_missing, rmse_observed]

    for i in range(predictions.shape[2]):
        slice_pred = predictions[:, :, i]
        slice_label = targets[:, :, i]
        slice_mask = mask[:, :, i]

        slice_mrmse, slice_irmse = [], []

        for j in range(len(slice_pred)):
            pred_row = slice_pred[j]
            label_row = slice_label[j]
            mask_row = slice_mask[j]

            valid = mask_row >= 0
            if np.sum(valid) == 0:
                continue

            min_val, max_val = label_row[valid].min(), label_row[valid].max()
            if max_val == min_val:
                continue

            obs_idx = mask_row == 0
            miss_idx = mask_row == 1

            if np.sum(obs_idx) > 0:
                rmse_obs = np.sqrt(np.mean(((pred_row[obs_idx] - label_row[obs_idx]) / (max_val - min_val)) ** 2))
                slice_irmse.append(rmse_obs)

            if np.sum(miss_idx) > 0:
                rmse_miss = np.sqrt(np.mean(((pred_row[miss_idx] - label_row[miss_idx]) / (max_val - min_val)) ** 2))
                slice_mrmse.append(rmse_miss)

        metrics_list.append(np.mean(slice_mrmse))
        metrics_list.append(np.mean(slice_irmse))

    metrics_arr = np.array(metrics_list)
    metrics_arr[0] = np.mean(metrics_arr[2:][::2])
    metrics_arr[1] = np.mean(metrics_arr[3:][::2])

    return metrics_arr

# ---------------------
# Model utilities
# ---------------------
def save_model_state(info_dict, filename='best.ckpt', folder=None):
    folder = folder or os.path.join(data_dir, 'models')
    os.makedirs(folder, exist_ok=True)

    model_state = {k: v.cpu() for k, v in info_dict['model'].state_dict().items()}
    torch.save({
        'epoch': info_dict['epoch'],
        'args': info_dict['args'],
        'best_metric': info_dict['best_metric'],
        'state_dict': model_state
    }, os.path.join(folder, filename))

def load_model_state(info_dict, filepath):
    checkpoint = torch.load(filepath)
    info_dict['epoch'] = checkpoint['epoch']
    info_dict['best_metric'] = checkpoint['best_metric']
    info_dict['model'].load_state_dict(checkpoint['state_dict'])

# ---------------------
# Metrics
# ---------------------
def compute_auc_score(true_labels, predictions):
    fpr, tpr, _ = metrics.roc_curve(true_labels, predictions)
    return metrics.auc(fpr, tpr)

# ---------------------
# CUDA helper
# ---------------------
def to_cuda(tensor, is_tensor=True):
    if args.gpu:
        return tensor.cuda(non_blocking=True) if is_tensor else tensor.cuda()
    return tensor

# ---------------------
# Learning rate scheduler
# ---------------------
def get_learning_rate(epoch):
    return args.lr  # Can extend for dynamic schedule if needed

# ---------------------
# Index-value mapping
# ---------------------
def map_to_index_value(data):
    if args.use_ve == 0:
        return Variable(to_cuda(data))
    data_np = data.numpy()
    idx = data_np // (args.split_num + 1)
    val = data_np % (args.split_num + 1)
    return [Variable(to_cuda(torch.from_numpy(idx.astype(np.int64)))),
            Variable(to_cuda(torch.from_numpy(val.astype(np.int64))))]

# ---------------------
# Training & evaluation
# ---------------------
def train_or_eval(loader, network, criterion, epoch, optimizer, best_metric, mode='train'):
    print(mode)
    lr = get_learning_rate(epoch)
    if mode == 'train':
        network.train()
        for g in optimizer.param_groups:
            g['lr'] = lr
    else:
        network.eval()

    losses, preds, labels, masks = [], [], [], []
    feature_mm_dict = myreadjson(os.path.join(data_dir, 'MIMIC_feature_mm_dict.json'))

    for batch in tqdm(loader):
        data, label, mask, files = batch[:4]
        data = map_to_index_value(data)

        # Optional neighbor & time data
        pre_input, pre_time, post_input, post_time, dd_list = batch[4:9]
        neib_data = [map_to_index_value(pre_input), Variable(to_cuda(pre_time)),
                     map_to_index_value(post_input), Variable(to_cuda(post_time))]
        dd_list = Variable(to_cuda(dd_list))

        label, mask = Variable(to_cuda(label)), Variable(to_cuda(mask))

        if args.model == 'tame':
            if args.dataset == 'MIMIC' and args.use_mm:
                output = network(data, neib=neib_data, dd=dd_list, mask=mask)
            elif args.use_ta:
                output = network(data, neib=neib_data, mask=mask)
            else:
                output = network(data, mask=mask)
        else:
            output = network(data, mask=mask)

        loss_val = criterion(output, label, mask)
        preds.append(output.data.cpu().numpy())
        losses.append(loss_val.data.cpu().numpy())
        labels.append(label.data.cpu().numpy())
        masks.append(mask.data.cpu().numpy())

        if mode == 'train':
            optimizer.zero_grad()
            loss_val.backward()
            optimizer.step()

    preds = np.concatenate(preds)
    labels = np.concatenate(labels)
    masks = np.concatenate(masks)
    metrics_arr = normalized_rmse(preds, labels, masks)
    avg_loss = np.mean(losses)

    print(f'\nEpoch {epoch} (lr={lr:.5f})')
    print(f'Loss: {avg_loss:.4f}')
    print(f'Metrics: {metrics_arr[:2]}')

    if mode == 'valid' and (best_metric[0] == 0 or metrics_arr[0] < best_metric[0]):
        best_metric = [metrics_arr[0], epoch]
        save_model_state({'args': args, 'model': network, 'epoch': epoch, 'best_metric': best_metric})

    return best_metric


Data Loader

In [2]:
import os
import json
import random
import numpy as np
import torch
from torch.utils.data import Dataset

# -----------------------------
# Utility Functions
# -----------------------------
def locate_index(value, reference_list, start=0, end=None):
    """Find the insertion index for a value in a sorted list."""
    end = end if end is not None else len(reference_list) - 1
    if value < reference_list[start]:
        return start
    if value > reference_list[end]:
        return end + 1
    if end - start == 1:
        return end
    mid = (start + end) // 2
    if value <= reference_list[mid]:
        return locate_index(value, reference_list, start, mid)
    else:
        return locate_index(value, reference_list, mid, end)

def filter_valid_lines(input_lines, output_lines):
    """Remove lines where all values are missing except for specific columns."""
    filtered_in, filtered_out = [], []
    for in_line, out_line in zip(input_lines, output_lines):
        presence = [0 if v in ['', 'NA'] else 1 for v in out_line.strip().split(',')]
        presence[0] = presence[6] = presence[8] = 0
        if sum(presence) > 0:
            filtered_in.append(in_line)
            filtered_out.append(out_line)
    return filtered_in, filtered_out

# -----------------------------
# Dataset Class
# -----------------------------
class EHRDataset(Dataset):
    def __init__(self, args, file_list, split='train'):
        assert split in ['train', 'valid', 'test']
        self.args = args
        self.split = split
        self.files = file_list

        self.feature_minmax = self._load_json('MIMIC_feature_mm_dict.json')
        self.feature_values = self._load_json(f'MIMIC_feature_value_dict_{args.split_num}.json')
        self.ehr_ids = self._build_ehr_ids('ehr_list.json')
        self.n_dd = 40

    def _load_json(self, filename):
        path = os.path.join(data_dir, filename)
        with open(path) as f:
            return json.load(f)

    def _build_ehr_ids(self, ehr_file):
        ehr_list = self._load_json(ehr_file)
        self.args.n_ehr = len(ehr_list) + 1
        return {ehr: idx + 1 for idx, ehr in enumerate(ehr_list)}

    # -----------------------------
    # Value Mapping Functions
    # -----------------------------
    def encode_input(self, val, feat_names, idx):
        """Map raw input value to embedding index or normalized float."""
        base_idx = (idx + 1) * (1 + self.args.split_num) + 1
        if val in ['NA', '']:
            return 0 if self.args.value_embedding == 'no' else base_idx - 1

        val = float(val)
        feat = feat_names[idx]

        if self.args.value_embedding == 'use_value':
            minv, maxv = self.feature_minmax[feat]
            scaled = (val - minv) / (maxv - minv + 1e-10)
            return int(scaled * self.args.split_num) + base_idx
        elif self.args.value_embedding == 'use_order':
            ref_vals = self.feature_values[feat][1:-1]
            return locate_index(val, ref_vals) + base_idx
        else:  # 'no'
            minv, maxv = self.feature_minmax[feat]
            scaled = (val - minv) / maxv + 1
            return round(scaled * self.args.split_num) / self.args.split_num

    def encode_output(self, val, feat_names, idx):
        if val in ['NA', '']:
            return 0
        val = float(val)
        minv, maxv = self.feature_minmax[feat_names[idx]]
        scaled = max(0, min((val - minv) / (maxv - minv), 1))
        return scaled

    # -----------------------------
    # Temporal Information
    # -----------------------------
    def extract_previous(self, data, line_idx, feat_names):
        reversed_data = data[::-1][:-1]
        return self.extract_future(reversed_data, len(data) - line_idx - 1, feat_names)

    def extract_future(self, data, line_idx, feat_names):
        data_slice = data[line_idx:]
        values, times = [0], [0]
        for col in range(1, len(data_slice[0])):
            val, t = '', 0
            for row in range(1, len(data_slice)):
                if data_slice[row][col] not in ['', 'NA']:
                    val = data_slice[row][col]
                    t = abs(int(data_slice[row][0]) - int(data_slice[0][0]))
                    break
            values.append(self.encode_input(val, feat_names, col))
            times.append(t)
        return values, times

    # -----------------------------
    # Pre-filling Missing Data
    # -----------------------------
    def prefill_missing(self, input_lines):
        input_matrix, mask_matrix = [], []
        lines_split = [line.strip().split(',') for line in input_lines]

        for i, line in enumerate(lines_split):
            mask = [1 if v not in ['', 'NA'] else 0 for v in line]
            mask_matrix.append(mask)
            input_matrix.append(line[:1])

        mask_matrix = np.array(mask_matrix)
        mask_matrix[0] = 0

        for col in range(1, mask_matrix.shape[1]):
            valid_rows = np.where(mask_matrix[:, col] > 0)[0]
            if len(valid_rows):
                mean_val = np.mean([float(lines_split[r][col]) for r in valid_rows])
            else:
                mean_val = 0
            last_val = mean_val
            for row in range(1, mask_matrix.shape[0]):
                if mask_matrix[row, col]:
                    input_matrix[row].append(lines_split[row][col])
                    last_val = lines_split[row][col]
                else:
                    input_matrix[row].append(str(last_val))
        return [','.join(x) for x in input_matrix]

    # -----------------------------
    # Data Item Retrieval
    # -----------------------------
    def get_mm_item(self, idx):
        input_file = self.files[idx]
        output_file = input_file.replace('with_missing', 'groundtruth')

        with open(output_file) as f:
            output_data = f.read().strip().split('\n')
        with open(input_file) as f:
            input_data = f.read().strip().split('\n')

        if self.args.use_ve == 0:
            input_data = self.prefill_missing(input_data)

        # Process per-line
        masks, inputs, outputs, pre_inputs, pre_times, post_inputs, post_times = [], [], [], [], [], [], []
        feat_names = input_data[0].strip().split(',')

        for line_idx, (inp_line, out_line) in enumerate(zip(input_data[1:], output_data[1:])):
            in_vals = inp_line.strip().split(',')
            out_vals = out_line.strip().split(',')

            mask_row, input_row, output_row = [], [], []
            for col_idx, (iv, ov) in enumerate(zip(in_vals, out_vals)):
                if ov in ['NA', '']:
                    mask_row.append(-1)
                    output_row.append(0)
                    input_row.append(self.encode_output(iv, feat_names, col_idx))
                else:
                    mask_row.append(0 if iv not in ['NA', ''] else 1)
                    output_row.append(self.encode_output(ov, feat_names, col_idx))
                    input_row.append(self.encode_input(iv, feat_names, col_idx) if self.args.use_ve else self.encode_output(iv, feat_names, col_idx))
            masks.append(mask_row)
            inputs.append(input_row)
            outputs.append(output_row)
            pre_in, pre_tm = self.extract_previous(input_data, line_idx + 1, feat_names)
            pre_inputs.append(pre_in)
            pre_times.append(pre_tm)
            post_in, post_tm = self.extract_future(input_data, line_idx + 1, feat_names)
            post_inputs.append(post_in)
            post_times.append(post_tm)

        # Convert to tensors
        def to_tensor(arr, dtype=np.float32):
            return torch.from_numpy(np.array(arr, dtype=dtype))

        return to_tensor(inputs), to_tensor(outputs), to_tensor(masks, np.int64), input_file, \
               to_tensor(pre_inputs, np.int64), to_tensor(pre_times, np.int64), \
               to_tensor(post_inputs, np.int64), to_tensor(post_times, np.int64), \
               torch.zeros((len(inputs), self.n_dd), dtype=torch.int64)

    # -----------------------------
    # Dataset Methods
    # -----------------------------
    def __getitem__(self, idx):
        if self.args.model in ['brnn', 'brits', 'mean', 'mice']:
            return self.get_mm_item(idx)  # simplified for example
        else:
            return self.get_mm_item(idx)

    def __len__(self):
        return len(self.files)


Loss

In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# -----------------------------
# Hard Example Mining
# -----------------------------
def select_hard_examples(outputs, targets, top_k, largest=True):
    """Select a subset of hardest examples based on predictions."""
    top_k = min(max(top_k, 10), len(outputs))
    _, indices = torch.topk(outputs, top_k, largest=largest)
    return outputs[indices], targets[indices]

# -----------------------------
# Binary Classification Loss
# -----------------------------
class BinaryLoss(nn.Module):
    def __init__(self, use_hard_mining=False):
        super(BinaryLoss, self).__init__()
        self.bce = nn.BCELoss()
        self.use_hard_mining = use_hard_mining

    def forward(self, predictions, targets):
        positive_mask = targets > 0.5
        negative_mask = targets <= 0.5

        pos_preds, pos_labels = predictions[positive_mask], targets[positive_mask]
        neg_preds, neg_labels = predictions[negative_mask], targets[negative_mask]

        pos_loss = neg_loss = 0.0

        # Hard mining
        if self.use_hard_mining:
            pos_preds, pos_labels = select_hard_examples(pos_preds, pos_labels, top_k=2, largest=False)
            neg_preds, neg_labels = select_hard_examples(neg_preds, neg_labels, top_k=6, largest=True)

        if len(pos_preds) > 0:
            pos_loss = 0.5 * self.bce(pos_preds, pos_labels)
        if len(neg_preds) > 0:
            neg_loss = 0.5 * self.bce(neg_preds, neg_labels)

        total_loss = pos_loss + neg_loss

        # Statistics
        predicted_class = (predictions.detach().cpu().numpy() > 0.5)
        target_class = targets.detach().cpu().numpy()
        pos_total = (target_class == 1).sum()
        neg_total = (target_class == 0).sum()
        pos_correct = ((predicted_class + target_class) == 2).sum()
        neg_correct = ((predicted_class + target_class) == 0).sum()

        return total_loss, pos_correct, pos_total, neg_correct, neg_total

# -----------------------------
# Mean Squared Error Loss
# -----------------------------
class MaskedMSELoss(nn.Module):
    def __init__(self, mode='missing'):
        super(MaskedMSELoss, self).__init__()
        assert mode in ['missing', 'init', 'both'], "Mode must be 'missing', 'init', or 'both'."
        self.mode = mode
        self.mse = nn.MSELoss()

    def forward(self, predictions, targets, mask):
        # Flatten all inputs
        predictions = predictions.view(-1)
        targets = targets.view(-1)
        mask = mask.view(-1)
        assert len(predictions) == len(targets) == len(mask)

        loss = 0.0

        # Missing values
        missing_idx = mask == 1
        if missing_idx.any():
            loss += self.mse(predictions[missing_idx], targets[missing_idx])

        # Initial (observed) values
        if self.mode == 'both':
            observed_idx = mask == 0
            if observed_idx.any():
                loss += self.mse(predictions[observed_idx], targets[observed_idx])

        return loss


In [50]:
import torch
import os
import sys
import time
import numpy as np
from sklearn import metrics
import random
import json
from glob import glob
from collections import OrderedDict
from tqdm import tqdm


import torch
from torch.autograd import Variable
from torch.backends import cudnn
from torch.nn import DataParallel
from torch.utils.data import DataLoader

class Args:
    def __init__(self,
                 epochs=100,
                 model='tame',
                 embed_size=512,
                 use_ve=1,
                 use_mm=1,
                 use_ta=1,
                 value_embedding='use_order'):

        # Base attributes
        self.dataset = 'MIMIC'
        self.n_code = 8

        self.n_visit=30
        self.nc = 4
        self.brnn = True
        self.random_missing = True
        self.split_num = 4000
        self.n_records = 30
        self.split_nor = 3
        self.loss = "both"
        self.num_layers = 2
        self.phase = "train"
        self.batch_size = 64
        self.resume = ""
        self.compute_weight = 0
        self.workers = 12
        self.lr = 0.001
        self.output_size = 27

        self.epochs = epochs
        self.model = model
        self.embed_size = embed_size
        self.rnn_size = embed_size       # same as embed_size
        self.hidden_size = embed_size    # same as embed_size

        # Flags provided by user
        self.use_ve = use_ve   # value embedding
        self.use_mm = use_mm   # min-max normalization
        self.use_ta = use_ta   # time-aware embedding
        self.value_embedding = value_embedding
        self.save_freq = 1
        self.save_pred_freq =10
        self.val_freq =1

        # GPU setup
        self.gpu = 1 if torch.cuda.is_available() else 0

        # Disable VE type when use_ve = 0
        if self.use_ve == 0:
            self.value_embedding = 'no'

        print('epochs,', self.epochs)

args = Args()
if torch.cuda.is_available():
    args.gpu = 1
else:
    args.gpu = 0
if args.use_ve == 0:
    args.value_embedding = 'no'

args.n_ehr = len(myreadjson(os.path.join(data_dir, 'ehr_list.json')))
args.name_list = myreadjson(os.path.join(data_dir, 'MIMIC_feature_list.json'))[1:]
# args.name_list.remove("c_reactive_protein")
print(args.name_list)
args.output_size = len(args.name_list)


epochs, 100
['wbc', 'bun', 'sodium', 'pt', 'inr', 'ptt', 'platelet', 'lactate', 'hemoglobin', 'glucose', 'chloride', 'creatinine', 'aniongap', 'bicarbonate', 'hematocrit', 'heartrate', 'resprate', 'tempc', 'meanbp', 'gcs_min', 'urineoutput', 'sysbp', 'diasbp', 'spo2', 'magnesium', 'c_reactive_protein', 'bands']


In [5]:
!cp -r drive/MyDrive/train_with_missing train_with_missing
!cp -r drive/MyDrive/bd4h/sort_pivoted/ train_groundtruth

In [24]:
!cp -r drive/MyDrive/bd4h/sort_pivoted/ train_groundtruth1

In [51]:
from glob import glob
from torch.utils.data import DataLoader

# -----------------------------
# Prepare file lists
# -----------------------------
all_files = sorted(glob(os.path.join('train_with_missing/*.csv')))
data_splits = myreadjson(os.path.join(data_dir, 'MIMIC_splits.json'))

train_files = [f for idx in range(0, 7) for f in data_splits[idx]]
valid_files = [f for idx in [7] for f in data_splits[idx]]
test_files = [f for idx in [8, 9] for f in data_splits[idx]]

if args.phase == 'test':
    train_phase, valid_phase, test_phase, train_shuffle = 'test', 'test', 'test', False
else:
    train_phase, valid_phase, test_phase, train_shuffle = 'train', 'valid', 'test', True

# -----------------------------
# Initialize datasets
# -----------------------------
train_dataset = EHRDataset(args, train_files, phase=train_phase)
valid_dataset = EHRDataset(args, valid_files, phase=valid_phase)
test_dataset = EHRDataset(args, test_files, phase=test_phase)

train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=train_shuffle,
                          num_workers=args.workers, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=args.batch_size, shuffle=False,
                          num_workers=args.workers, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False,
                         num_workers=args.workers, pin_memory=True)

args.vocab_size = (args.output_size + 2) * (1 + args.split_num) + 5

# -----------------------------
# Initialize model and loss
# -----------------------------
net = NeuralNet(args)
loss_fn = MaskedMSELoss(args)  # updated class name

net = to_cuda(net, 0)
loss_fn = to_cuda(loss_fn, 0)

best_metric = [0, 0]
start_epoch = 0

# -----------------------------
# Optimizer
# -----------------------------
optimizer = torch.optim.Adam(net.parameters(), lr=args.lr)

# -----------------------------
# Training / Testing loop
# -----------------------------
if args.phase == 'train':
    for epoch in range(start_epoch, args.epochs):
        print('Starting epoch:', epoch)
        train_or_eval(train_loader, net, loss_fn, epoch, optimizer, best_metric)
        best_metric = train_or_eval(valid_loader, net, loss_fn, epoch, optimizer, best_metric, phase='valid')

elif args.phase == 'test':
    folder = os.path.join(data_dir, 'imputation_result')
    os.system(f'rm -r {folder}')
    os.system(f'mkdir {folder}')

    train_or_eval(train_loader, net, loss_fn, 0, optimizer, best_metric, phase='test')
    train_or_eval(valid_loader, net, loss_fn, 0, optimizer, best_metric, phase='test')
    train_or_eval(test_loader, net, loss_fn, 0, optimizer, best_metric, phase='test')


start epoch : 0
train


100%|██████████| 129/129 [03:16<00:00,  1.53s/it]



Train Epoch 000 (lr 0.00100)
loss: 0.0396 	
metric: 1.6763	2.3268
wbc.....................................1.1814......1.3521
bun.....................................1.7697......1.9306
sodium..................................0.9338......1.0512
pt......................................4.1365......4.4140
inr.....................................3.0803......3.3606
ptt.....................................2.9493......2.8765
platelet................................1.3169......1.4584
lactate.................................2.0069......2.1177
hemoglobin..............................0.7494......0.8325
glucose.................................0.5665......0.6372
chloride................................0.7890......0.8966
creatinine..............................2.3843......2.6127
aniongap................................0.6861......0.7548
bicarbonate.............................0.8372......0.9104
hematocrit..............................0.7150......0.7888
heartrate...............................0.5506..

100%|██████████| 19/19 [00:29<00:00,  1.58s/it]
/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)



Train Epoch 000 (lr 0.00100)
loss: 0.0209 	
metric: nan	2.9490
wbc.....................................0.7609......0.9920
bun.....................................1.3366......1.5862
sodium..................................0.5975......0.6576
pt......................................2.5084......2.7346
inr.....................................0.9009......1.0574
ptt.....................................2.0492......2.7497
platelet................................0.9711......1.2167
lactate.................................0.6486......0.9923
hemoglobin..............................0.5573......0.6972
glucose.................................0.3611......0.4306
chloride................................0.5631......0.6561
creatinine..............................2.0908......2.1641
aniongap................................0.4970......0.6424
bicarbonate.............................0.6409......0.6912
hematocrit..............................0.5775......0.6491
heartrate...............................0.4791.....

...


Train Epoch 035 (lr 0.00100)
loss: 0.0044 	
metric: 0.8538	0.8324
wbc.....................................0.5279......0.6194
bun.....................................0.4634......0.5390
sodium..................................0.3029......0.2926
pt......................................1.4340......1.3023
inr.....................................0.8964......0.8598
ptt.....................................1.2088......0.9620
platelet................................0.5017......0.5377
lactate.................................0.8475......0.7707
hemoglobin..............................0.2291......0.1973
glucose.................................0.2861......0.2114
chloride................................0.2321......0.2222
creatinine..............................0.6286......0.7135
aniongap................................0.2699......0.2228
bicarbonate.............................0.2593......0.2179
hematocrit..............................0.2241......0.1908
heartrate...............................0.2039..

100%|██████████| 19/19 [00:18<00:00,  1.04it/s]



Train Epoch 035 (lr 0.00100)
loss: 0.0039 	
metric: nan	1.8499
wbc.....................................0.4461......0.6212
bun.....................................0.3506......0.4217
sodium..................................0.2516......0.2146
pt......................................1.2378......1.0262
inr.....................................1.0233......0.8888
ptt.....................................0.5896......0.7759
platelet................................0.3172......0.4302
lactate.................................0.4597......0.6339
hemoglobin..............................0.2217......0.2035
glucose.................................0.2492......0.1641
chloride................................0.2529......0.1880
creatinine..............................0.5090......0.5801
aniongap................................0.2869......0.2263
bicarbonate.............................0.2382......0.1690
hematocrit..............................0.2283......0.1904
heartrate...............................0.1979.....

 38%|███▊      | 49/129 [00:59<01:36,  1.21s/it]


KeyboardInterrupt: 

In [20]:
import os
os.path.exists("train_groundtruth/176368.csv")


False